In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

tweets = pd.read_csv('csv/train.csv', usecols=['location', 'target'])
tweets.head()

,location,target
0,NaN,1
1,NaN,1
2,NaN,1
3,NaN,1
4,NaN,1


In [2]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   location  5080 non-null   object
 1   target    7613 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.1+ KB


In [3]:
# Con esto podemos ver que Target no tiene valores nulos pero que location si (alrededor de 2550 valores nulos)

In [7]:
tweets_by_loc = tweets.set_index('location')
tweets_by_loc.head()

,target
location,
NaN,1
NaN,1
NaN,1
NaN,1
NaN,1


In [26]:
tweets['cuantity'] = 1
tweets_by_loc= tweets.groupby('location').sum()

In [27]:
tweets_by_loc.head()

,target,cuantity
location,,
,1,1
Glasgow,0,1
"Melbourne, Australia",0,1
News,1,1
å_,0,1


In [28]:
tweets_by_loc.sort_values('target', ascending=False)

,target,cuantity
location,,
USA,67,104
United States,27,50
Nigeria,22,28
India,20,24
Mumbai,19,22
...,...,...
"Not where I want to be, yet",0,1
Not Steven Yeun / AMC.,0,1
"Clearwater, FL",0,1
